# Assignment 1
---

Firstly, we import several modules that will be used for the process

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

As scraping data from website will be tedious, we create function to get the soup easily

In [2]:
def get_soup(url):
    soup_source = requests.get(url).text
    
    return BeautifulSoup(soup_source,'lxml')

Then, we will go to DR NTU website and list out professors that are in the list and try to scrape several information from there. However, when we look at the website, the list of professors are listed in two page. There is an idea to automate the process by finding the "next" reference, but looks like the reference page is not the same so we will do it in primitive way, which by plug in the website one by one

In [3]:
prof_list_URL1 = "https://dr.ntu.edu.sg/simple-search?filterquery=ou00030&filtername=school&filtertype=authority&location=researcherprofiles&sort_by=bi_sort_4_sort&order=ASC"
prof_list_URL2 = "https://dr.ntu.edu.sg/simple-search?query=&location=researcherprofiles&filter_field_1=school&filter_type_1=authority&filter_value_1=ou00030&crisID=&relationName=&sort_by=bi_sort_4_sort&order=asc&rpp=50&etal=0&start=50"

soup_prflist1 = get_soup(prof_list_URL1)
soup_prflist2 = get_soup(prof_list_URL2)

Looks like from this we can get two more information which is <b> DR NTU url </b> and <b> Email </b>

In [4]:
names1 = [x.text for x in soup_prflist1.find("div", class_="discovery-result-results").find_all('a')][1:]
links1 = ["https://dr.ntu.edu.sg" + x.get('href') for x in soup_prflist1.find("div", class_="discovery-result-results").find_all('a')][1:]
emails1 = [x.text for x in soup_prflist1.find_all("td", headers="t3")]

In [5]:
names2 = [x.text for x in soup_prflist2.find("div", class_="discovery-result-results").find_all('a')][1:]
links2 = ["https://dr.ntu.edu.sg" + x.get('href') for x in soup_prflist2.find("div", class_="discovery-result-results").find_all('a')][1:]
emails2 = [x.text for x in soup_prflist2.find_all("td", headers="t3")]

Next, we append the name together the two website

In [6]:
names = names1 + names2
links = links1 + links2
emails = emails1 + emails2

For Website URL, we will use the one that listed in his/her DR-NTU URL. To do this, firstly we have to go to his/her website and scrape the personal website url, which located beside the profile picture under name "Personal Website"

In [7]:
website = []

for url in links:
    soup_tmp = get_soup(url)
    soup_tmp = get_soup(url)
    if soup_tmp.find("div",id = "personalsiteDiv"):
        webs = [x.get('href') for x in soup_tmp.find("div",id = "personalsiteDiv").find_all('a')][0]
    else:
        webs = np.nan
    website.append(webs)

Next, we will lookout for the DBLP URL and Citations from Google Scholar. But firstly, we need to modify some of professors name because some of them have different formatting of name for DBLP and Google Scholar.

In [8]:
prof_change = {'Jagath Chandana Rajapakse' : 'Jagath C Rajapakse',
              'Lana Obraztsova' : 'Svetlana Obraztsova',
              'Ke Yiping, Kelly' : 'Ke Yiping',
              'Sourav Saha Bhowmick' : 'Sourav S Bhowmick',
              'Tay Kian Boon' : 'Tay K Boon',
              'Douglas Leslie Maskell' : 'Douglas L Maskell',
              'Joty Shafiq Rayhan' : 'Joty Shafiq',
              'Luke Ong （翁之昊）': 'Luke Ong',
              'Luo Siqiang （骆思强）' : 'Luo Siqiang',
              'Quek Hiok Chai' : 'Quek Chai'}

For DBLP URL, we will use the search to find out the website. When we search we look out for the professor's profile that will be seen on the left one. Then we take the website which is the one that it refers

In [9]:
url_dblp = []

for name in names:
    if name in prof_change.keys():
        name_tmp = prof_change[name]
        name = name_tmp
    name1 = name.replace(",","")
    name2 = name1.replace(" ","+")
    url = "https://dblp.org/search?q=" + name2
    soup_tmp = get_soup(url)
    if soup_tmp.find("ul",class_ = "result-list"):
        url_prof = [x.get('href') for x in soup_tmp.find("ul",class_ = "result-list").find_all('a')][0]
    else:
        url_prof = np.nan
    url_dblp.append(url_prof)

For the Google Scholar, to find the profile is the same and next, after we go to the profile we look out for the table on top right of the page and take the number of Citations from there

In [10]:
citations = []

for name in names:
    if name in prof_change.keys():
        name_tmp = prof_change[name]
        name = name_tmp
    name1 = name.replace(",","")
    name2 = name1.replace(" ","+")
    
    url = "https://scholar.google.com/citations?view_op=search_authors&hl=en&mauthors=" + name2 + "&btnG="
    soup_tmp1 = get_soup(url)
    if soup_tmp1.find("div",class_ = "gs_ai_t"):
        url_prof = "https://scholar.google.com" + [x.get('href') for x in soup_tmp1.find("div",class_ = "gs_ai_t").find_all('a')][0]
        soup_tmp2 = get_soup(url_prof)
        table = soup_tmp2.find("table", id = "gsc_rsb_st")
        row = table.find_all("tr")[1]
        col = row.find_all("td")
        col = [ele.text.strip() for ele in col]
        cit = int(col[1])
    else:
        cit = np.nan
    citations.append(cit)

Now let's put it together all of them into one dataframe and save it to csv file

In [11]:
myDict = {}
myDict['Full Name'] = names
myDict['Email'] = emails
myDict['DR-NTU URL'] = links
myDict['Website URL'] = website
myDict['DBLP URL'] = url_dblp
myDict['Citations (All)'] = citations

df = pd.DataFrame(myDict)
df.to_csv("Andrew_Wiraatmaja.csv")